<a href="https://colab.research.google.com/github/EthanXC/RedditProject/blob/main/RedditProject1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install datasets
# FYI the accellerate -u is very important to avoid version issues with torch
!pip install transformers
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-

In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("fddemarco/pushshift-reddit-comments", streaming = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [5]:
print(dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['author', 'body', 'controversiality', 'created_utc', 'link_id', 'score', 'subreddit', 'subreddit_id', 'id'],
        n_shards: 50
    })
})


In [6]:
it = iter(dataset['train'])


In [7]:
extract_data = next(it)
#print(extract_data)

print(extract_data['body'])
#testing if we can get a sample comment

isn ' t this against the first amendment ? doesn ' t the first amendment give us the right to assemble and protest ?


In [8]:
data = {
    'column1_name': ['text'],
    'column2_name': ['score'],
    'column3_name': ['labels']
}

df = pd.DataFrame(data)

In [9]:
# Initialize an empty DataFrame with columns
df = pd.DataFrame(columns=['text', 'score', 'labels'])

nsfw_keywords = ['nsfw', 'adult', 'sex', 'porn', 'xxx', 'gonewild', 'nudes', 'slut', 'WTF',
'hentai', 'cum', 'rule34', 'pussy', 'onlyfans', 'milf', 'nude', 'boob']

subreddit_counts = {}

row_variable = 10000 # only 10,000 rows

for i in range(row_variable):
    extract_data = next(it)
    text = extract_data['body']
    score = extract_data['score']
    labels = extract_data['subreddit']

    if any(keyword in labels.lower() for keyword in nsfw_keywords):
      continue

     # Check and update subreddit count
    if labels in subreddit_counts:
        if subreddit_counts[labels] >= (0.005*row_variable):
            continue  # Skip this subreddit if it has already 500 entries
        subreddit_counts[labels] += 1
    else:
        subreddit_counts[labels] = 1

    new_row_data = {'text': text, "score": score, "labels": labels}
    df.loc[len(df)] = new_row_data

    if i % 1000 == 0:
        print(i)
        print(new_row_data)





0
{'text': 'oil .', 'score': -4, 'labels': 'AskReddit'}
1000
{'text': 'is that a credible list ? how did they obtain their data ?', 'score': 1, 'labels': 'starcraft'}
2000
{'text': 'ok then : would you rather live in a street full of mps , or a street full of al - qaeda members , since this video seems to be implying that the threat level is the same .', 'score': -2, 'labels': 'unitedkingdom'}
3000
{'text': 'r / bad _ cop _ no _ latte coming soon .', 'score': 24, 'labels': 'Bad_Cop_No_Donut'}
4000
{'text': 'amen', 'score': 3, 'labels': 'treecomics'}
5000
{'text': "i remember that . . . it was a form message , wasn ' t it ? i used to end every one of my gm conversations , when they asked me ' is there anything else i can do to help you ' , with something along the lines of ' i know this is a shot in the dark , but could you possibly tell me an email address with which i might submit feedback and them promptly close the ticket , disallowing further communication with you ? ' i figured i 

In [10]:
df.head()

,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [11]:
id2label = dict(enumerate(set(df['labels'])))

In [12]:
id2label

{0: 'shittyaskscience',
 1: 'LearnJapanese',
 2: 'tipofmytongue',
 3: 'Portal',
 4: 'Cyberpunk',
 5: 'glasgow',
 6: 'treecomics',
 7: 'tf2items',
 8: 'askdrugs',
 9: 'MMFB',
 10: 'LeagueofLegendsMeta',
 11: 'gamernews',
 12: 'Maplestory',
 13: 'italy',
 14: 'slackerrecipes',
 15: 'spacedicks',
 16: 'ForeverAloneDating',
 17: 'Marvel',
 18: 'Etsy',
 19: 'Assistance',
 20: 'bulgaria',
 21: 'Graffiti',
 22: 'cordcutters',
 23: 'mead',
 24: 'ancientworldproblems',
 25: 'futurebeats',
 26: 'wallpaper',
 27: 'AskReddit',
 28: 'jobs',
 29: 'keto',
 30: 'leagueoflegends',
 31: 'Slender_Man',
 32: 'JRPG',
 33: 'StarWars',
 34: 'Yakima',
 35: 'washingtondc',
 36: 'nyjets',
 37: 'FixedGearBicycle',
 38: 'skateboarding',
 39: 'Objectivism',
 40: 'WeAreTheMusicMakers',
 41: 'classicrage',
 42: 'Frugal',
 43: 'Silverbugs',
 44: 'nfl',
 45: 'cincinnati',
 46: 'masculism',
 47: 'Rateme',
 48: 'obama',
 49: 'editing',
 50: 'Lubbock',
 51: 'learnprogramming',
 52: 'SteamGameSwap',
 53: 'r4r',
 54: 'comi

In [ ]:
df.head()


,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load roberta model and tokenizer filling out the code below based on the documentation HINT: Click on on use in transformers and the code will be there
#model_name = "Roberta model"# code unnessisary but I like to use it to store the string name to make the code cleaner
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = len(id2label))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
nan_rows = df[df.isna().T.any().T]
nan_rows.head()

,text,score,labels


In [ ]:
null_rows = df[df.isnull().any(axis=1)]
null_rows.head()

,text,score,labels


In [ ]:
if np.isinf(df['score']).any():
  print("HELLO")

In [ ]:
print(df.dtypes)

text      object
score      int64
labels    object
dtype: object


In [ ]:
df.head()

,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [ ]:
from sklearn.model_selection import train_test_split

#TODO: Complete this line using the train_test_split function and set the test size to .21
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['labels'], test_size=0.21)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
# Reset the index
train_texts, train_labels = train_texts.reset_index(drop=True), train_labels.reset_index(drop=True)
val_texts, val_labels = val_texts.reset_index(drop=True), val_labels.reset_index(drop=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, return_tensors = "pt")
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, return_tensors = "pt")

In [ ]:
id2label

{0: 'bestof',
 1: 'PoliticalDiscussion',
 2: 'skateboarding',
 3: 'bulgaria',
 4: 'EngineeringStudents',
 5: 'Parenting',
 6: 'twincitiessocial',
 7: 'ideasfortheadmins',
 8: 'toycameras',
 9: 'Mommit',
 10: 'resumes',
 11: 'engineering',
 12: 'RandomActsOfPizza',
 13: 'philadelphia',
 14: 'soccer',
 15: 'beer',
 16: 'realdubstep',
 17: 'amiugly',
 18: 'firefox',
 19: 'CoffeeWithJesus',
 20: 'Republican',
 21: 'comics',
 22: 'nba',
 23: 'chile',
 24: 'londonontario',
 25: 'nyjets',
 26: 'eu3',
 27: 'torrents',
 28: 'happy',
 29: 'Ubuntu',
 30: 'NetflixBestOf',
 31: 'Urbandead',
 32: 'hiphopheads',
 33: 'thewalkingdead',
 34: 'Yakima',
 35: 'MDMA',
 36: 'RandomKindness',
 37: 'PHXMeetup',
 38: 'Cyberpunk',
 39: 'DebateAChristian',
 40: 'Games',
 41: 'minnesota',
 42: 'FolkPunk',
 43: 'gentlemanboners',
 44: 'haskell',
 45: 'bf3pc',
 46: 'GalaxyNexus',
 47: 'GaymersGoneMild',
 48: 'indianapolis',
 49: 'fashion',
 50: 'detroitlions',
 51: 'breakingbad',
 52: 'space',
 53: 'wicked_edge',
 

In [ ]:
id2label_reversed = {v: k for k, v in id2label.items()}

In [ ]:
id2label_reversed #word:number

{'bestof': 0,
 'PoliticalDiscussion': 1,
 'skateboarding': 2,
 'bulgaria': 3,
 'EngineeringStudents': 4,
 'Parenting': 5,
 'twincitiessocial': 6,
 'ideasfortheadmins': 7,
 'toycameras': 8,
 'Mommit': 9,
 'resumes': 10,
 'engineering': 11,
 'RandomActsOfPizza': 12,
 'philadelphia': 13,
 'soccer': 14,
 'beer': 15,
 'realdubstep': 16,
 'amiugly': 17,
 'firefox': 18,
 'CoffeeWithJesus': 19,
 'Republican': 20,
 'comics': 21,
 'nba': 22,
 'chile': 23,
 'londonontario': 24,
 'nyjets': 25,
 'eu3': 26,
 'torrents': 27,
 'happy': 28,
 'Ubuntu': 29,
 'NetflixBestOf': 30,
 'Urbandead': 31,
 'hiphopheads': 32,
 'thewalkingdead': 33,
 'Yakima': 34,
 'MDMA': 35,
 'RandomKindness': 36,
 'PHXMeetup': 37,
 'Cyberpunk': 38,
 'DebateAChristian': 39,
 'Games': 40,
 'minnesota': 41,
 'FolkPunk': 42,
 'gentlemanboners': 43,
 'haskell': 44,
 'bf3pc': 45,
 'GalaxyNexus': 46,
 'GaymersGoneMild': 47,
 'indianapolis': 48,
 'fashion': 49,
 'detroitlions': 50,
 'breakingbad': 51,
 'space': 52,
 'wicked_edge': 53,
 

In [ ]:
import torch as pt

class SpamDataset(pt.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values

    def __getitem__(self, idx):
        item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels) > 0:  # Check if labels array has any elements
            labels = self.labels[idx]
            print(labels)
            labels_converted = id2label_reversed[labels]
            print(labels_converted)
            item["labels"] = pt.tensor(labels_converted)
        return item
    def __len__(self):
        print(len(self.labels))
        return len(self.labels)

def list_of_dicts_to_dict_of_lists(d):
  dic = d[0]
  keys = dic.keys()
  values = [dic.values() for dic in d]
  return {k: list(v) for k,v in zip(keys, zip(*values))}

In [ ]:
Train_Dataset = SpamDataset(train_encodings, train_labels)


In [ ]:
Val_Dataset = SpamDataset(val_encodings, val_labels)

In [ ]:
!pip install datasets transformers torch evaluate nltk rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f41d139f061e408bde9340d189232b21012c1ed512b02e23292494374bf002a9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate


In [ ]:
from sklearn.metrics import accuracy_score

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
#%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
#%env CUDA_LAUNCH_BLOCKING

'1'

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs, let's try 1 for now
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = Train_Dataset,
    eval_dataset = Val_Dataset,
    compute_metrics = compute_metrics
    #TODO: figure out what 5 params we need here. There should be 5 and the last is how we compute the metrics later . . .
)

7751


In [ ]:
#set(id2label_reversed.values)

In [ ]:
# TODO: Train the model (HINT: this is 1 short line of code just calling the trainer)
trainer.train()

7751
7751
7751
7751
7751
7751
7751
7751
7751
7751
7751
7751
7751
Diablo
206
atheism
575
Eve
492
funny
842
pics
421
politics
251
funny
842
magicTCG
431
loseit
545
AskReddit
361
AskReddit
361
doctorwho
798
politics
251
mylittlepony
786
gif
229
AdviceAnimals
711


<ipython-input-25-df3c2ef05379>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,6.717800
20,6.694500
30,6.722500
40,6.690300
50,6.678800
60,6.635800
70,6.598700
80,6.553500
90,6.419800
100,6.386200


Streaming output truncated to the last 5000 lines.
AskReddit
361
soccer
14
Minecraft
178
politics
251
gaming
394
funny
842
pics
421
AskReddit
361
IAmA
134
tipofmytongue
558
india
283
londonontario
24
AskReddit
361
politics
251
Christianity
418
AskReddit
361
orlando
685
movies
187
guns
686
trees
256
politics
251
leagueoflegends
202
pokemon
270
trees
256
IAmA
134
gameswap
473
AskReddit
361
leagueoflegends
202
comicbooks
395
trees
256
AdviceAnimals
711
DebateReligion
350
scifi
579
Volkswagen
734
WTF
133
soccer
14
fffffffuuuuuuuuuuuu
184
leagueoflegends
202
AdviceAnimals
711
atheism
575
MLPLounge
407
politics
251
funny
842
AskReddit
361
fitnesscirclejerk
780
atheism
575
gaming
394
OkCupid
678
trees
256
atheism
575
politics
251
DoesAnybodyElse
54
runescape
656
politics
251
iphone
848
NoFap
645
leagueoflegends
202
creepy
165
worldnews
80
AskReddit
361
LosAngeles
703
gaming
394
movies
187
funny
842
treemusic
513
AskReddit
361
MLPLounge
407
TwoXChromosomes
287
boardgames
825
aww
668
skyrim
512

<ipython-input-25-df3c2ef05379>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


Streaming output truncated to the last 5000 lines.
807
nfl
812
gaming
394
gaming
394
AskReddit
361
videos
310
starcraft
509
WTF
133
politics
251
foreveraloneteens
828
CollegeBasketball
755
CFB
324
MLPLounge
407
fffffffuuuuuuuuuuuu
184
fffffffuuuuuuuuuuuu
184
AskReddit
361
funny
842
AskReddit
361
BorderCollie
614
pics
421
politics
251
SteamGameSwap
777
AskWomen
316
pics
421
bf3pc
45
AskReddit
361
fffffffuuuuuuuuuuuu
184
worldwar
633
askseddit
399
unt
820
funny
842
funny
842
starcraft
509
sewing
744
depression
212
AskReddit
361
pics
421
politics
251
baseball
504
fffffffuuuuuuuuuuuu
184
atheism
575
politics
251
amiugly
17
atheism
575
nfl
812
pics
421
funny
842
vfx
74
politics
251
Fallout
660
alternativeart
363
funny
842
Gore
459
harrypotter
640
baseball
504
leagueoflegends
202
comicbooks
395
politics
251
exmormon
155
AskReddit
361
unitedkingdom
204
TwoXChromosomes
287
nfl
812
atheism
575
buildapc
58
canada
816
AndroidQuestions
274
discgolf
802
AskReddit
361
WTF
133
trees
256
wow
345
houst

TrainOutput(global_step=969, training_loss=5.145474161279952, metrics={'train_runtime': 435.3672, 'train_samples_per_second': 17.803, 'train_steps_per_second': 2.226, 'total_flos': 2054937897308160.0, 'train_loss': 5.145474161279952, 'epoch': 1.0})

In [ ]:
#TODO use the evaluate method on the trainer to get and print the results. Feel free to look at Huggingface Docs
results = trainer.evaluate()

2061
2061
2061
2061
AskStatistics
65
AskReddit
361
MinecraftInventions
796
AskReddit
361
atheism
575
gaming
394
politics
251
Terraria
829
WTF
133
torrents
27
Games
40
gaming
394
leagueoflegends
202
SteamGameSwap
777
AskReddit
361
funny
842
pics
421
MMA
635
OneParagraph
491
zelda
799
SteamGameSwap
777
fffffffuuuuuuuuuuuu
184
ShitRedditSays
708
Minecraft
178
AskReddit
361
pics
421
Conservative
477
sanfrancisco
767
snowboarding
85
politics
251
funny
842
gaming
394
AskReddit
361
fffffffuuuuuuuuuuuu
184
AskReddit
361
WTF
133
politics
251
unitedkingdom
204
funny
842
Music
662
exmormon
155
AskReddit
361
knives
201
gaming
394
ShitRedditSays
708
SteamTrade
807
politics
251
truegaming
769
AskReddit
361
UTAustin
337
IAmA
134
SteamGameSwap
777
offbeat
818
AskReddit
361
editing
347
AskReddit
361
vancouver
62
AdviceAnimals
711
Android
318
SteamTrade
807
guns
686
Mommit
9
politics
251
AdviceAnimals
711


<ipython-input-25-df3c2ef05379>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


2061
2061


funny
842
gameswap
473
pics
421
Ijustwatched
296
pics
421
DebateAChristian
39
WTF
133
SteamTrade
807
nfl
812
AskReddit
361
AskReddit
361
Meditation
102
DoesAnybodyElse
54
pics
421
AskReddit
361
soccer
14
aww
668
pokemon
270
gaming
394
trees
256
AskReddit
361
Christianity
418
Buddhism
570
atheism
575
gaming
394
Shemales
173
pics
421
AskReddit
361
funny
842
atheism
575
gaming
394
atheism
575
2061
politics
251
nfl
812
guns
686
politics
251
starcraft
509
audiophile
609
WTF
133
AskReddit
361
AskReddit
361
AskReddit
361
funny
842
IDAP
529
AskReddit
361
gaming
394
cigars
328
mw3
778
IAmA
134
Art
253
AskReddit
361
politics
251
Games
40
starcraft
509
gaming
394
AskReddit
361
mylittlepony
786
funny
842
gaming
394
politics
251
DotA2
602
swtor
791
fitnesscirclejerk
780
exmormon
155
2061
AskReddit
361
bestofthefray
461
Minecraft
178
TwoXChromosomes
287
comics
21
knives
201
nfl
812
fffffffuuuuuuuuuuuu
184
AdviceAnimals
711
hookah
571
SteamGameSwap
777
hockey
198
gaming
394
Philippines
90
videos
310


In [ ]:
# TODO: Save the model
trainer.save_model('my_model')

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
# TODO: Now to finish off I want you to load the model you trained and saved and write a fake spam email for it mthen have the model classify it
model = AutoModelForSequenceClassification.from_pretrained("my_model")
msg = "I want to download minecraft mods"
msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")

outputs = model(**msg_encodings)
probabilities = torch.softmax(outputs.logits[0], dim = 0 )
sort_prob = probabilities.sort(descending = True)
print(f"outputs: {outputs}")




outputs: SequenceClassifierOutput(loss=None, logits=tensor([[-1.8448, -0.9490, -3.3585, -4.1030, -2.0015, -1.0623, -2.5766, -2.6509,
         -3.8563, -4.1578, -2.6703, -1.8623, -3.5851, -3.8845,  0.2382, -0.6976,
         -2.6611,  0.2561, -3.8731, -2.3820, -3.7609, -0.8236,  0.2139, -3.7164,
         -2.3364, -4.0861, -3.4772, -1.4665, -3.7071, -1.7744, -1.7096, -2.3748,
         -0.9848, -3.6439, -2.5600, -2.7642, -1.9238, -2.7035, -3.3323, -0.9724,
          1.0058, -3.5625, -2.4345, -1.6525, -2.2728, -2.7192, -3.0703, -2.4582,
         -2.2019, -4.2738, -3.6544, -2.2320, -2.2036, -1.2424, -1.1342, -1.6158,
         -4.0018, -2.5195,  0.6504, -2.8834, -2.4655, -2.1987, -0.9067, -1.7431,
         -2.6560, -3.8127, -2.7563, -0.8069, -1.0059, -3.0439, -3.5598, -2.3434,
         -4.1856, -3.4992, -3.4342, -4.0028, -2.2760, -3.7559, -4.0328, -2.4832,
          1.3704, -0.4041, -2.6065, -2.6295, -1.8074, -1.3962, -1.2937, -3.0231,
         -3.2864, -3.5674, -3.6392, -2.4363,  0.9169, -2.

In [ ]:
outputs.logits.shape

torch.Size([1, 852])

In [ ]:
len(id2label)

852

In [ ]:
probabilities = torch.softmax(outputs.logits[0], dim = 0 )
print(probabilities)

tensor([3.3660e-04, 8.2447e-04, 7.4088e-05, 3.5187e-05, 2.8780e-04, 7.3611e-04,
        1.6193e-04, 1.5033e-04, 4.5032e-05, 3.3311e-05, 1.4744e-04, 3.3077e-04,
        5.9067e-05, 4.3784e-05, 2.7025e-03, 1.0601e-03, 1.4880e-04, 2.7513e-03,
        4.4286e-05, 1.9671e-04, 4.9541e-05, 9.3461e-04, 2.6376e-03, 5.1796e-05,
        2.0589e-04, 3.5787e-05, 6.5793e-05, 4.9137e-04, 5.2282e-05, 3.6118e-04,
        3.8532e-04, 1.9814e-04, 7.9542e-04, 5.5694e-05, 1.6464e-04, 1.3422e-04,
        3.1104e-04, 1.4262e-04, 7.6050e-05, 8.0535e-04, 5.8229e-03, 6.0412e-05,
        1.8664e-04, 4.0799e-04, 2.1941e-04, 1.4040e-04, 9.8836e-05, 1.8228e-04,
        2.3553e-04, 2.9665e-05, 5.5107e-05, 2.2855e-04, 2.3512e-04, 6.1483e-04,
        6.8506e-04, 4.2323e-04, 3.8936e-05, 1.7145e-04, 4.0811e-03, 1.1914e-04,
        1.8096e-04, 2.3628e-04, 8.6009e-04, 3.7265e-04, 1.4956e-04, 4.7042e-05,
        1.3530e-04, 9.5032e-04, 7.7882e-04, 1.0147e-04, 6.0576e-05, 2.0444e-04,
        3.2399e-05, 6.4363e-05, 6.8681e-

In [ ]:
sort_prob = probabilities.sort(descending = True)

In [ ]:
sort_prob.values[0].item()



0.12538181245326996

In [ ]:
print(sort_prob)

torch.return_types.sort(
values=tensor([1.2538e-01, 8.1174e-02, 5.2343e-02, 5.1015e-02, 4.4945e-02, 2.6012e-02,
        2.4828e-02, 2.3687e-02, 2.2188e-02, 1.9061e-02, 1.8652e-02, 1.8078e-02,
        1.6827e-02, 1.4841e-02, 1.3871e-02, 1.2454e-02, 1.2238e-02, 9.2823e-03,
        8.3847e-03, 7.7276e-03, 7.3659e-03, 7.0917e-03, 6.7289e-03, 6.2498e-03,
        6.0591e-03, 5.9430e-03, 5.8229e-03, 5.3275e-03, 5.1919e-03, 4.2702e-03,
        4.1498e-03, 4.0940e-03, 4.0811e-03, 3.9216e-03, 3.8612e-03, 3.6648e-03,
        3.5764e-03, 3.3213e-03, 3.3179e-03, 3.1381e-03, 2.8862e-03, 2.7888e-03,
        2.7749e-03, 2.7513e-03, 2.7025e-03, 2.6793e-03, 2.6400e-03, 2.6376e-03,
        2.5522e-03, 2.3606e-03, 2.3501e-03, 2.2967e-03, 2.2058e-03, 2.0547e-03,
        2.0192e-03, 2.0192e-03, 1.8720e-03, 1.8674e-03, 1.8579e-03, 1.8402e-03,
        1.8062e-03, 1.8013e-03, 1.7850e-03, 1.7383e-03, 1.6944e-03, 1.6703e-03,
        1.6148e-03, 1.6043e-03, 1.5991e-03, 1.5882e-03, 1.5782e-03, 1.5759e-03,
        

In [ ]:
def print_out_top_x_probs(x,phrase):
  msg = phrase
  sort_prob.values[0].item()
  print('Here are the top 10 most subreddits with similar')
  model = AutoModelForSequenceClassification.from_pretrained("my_model")
  msg = "I want to download minecraft mods"
  msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")

  outputs = model(**msg_encodings)
  probabilities = torch.softmax(outputs.logits[0], dim = 0 )
  sort_prob = probabilities.sort(descending = True)
  for n in range(x):
    sort_probs_iter = sort_prob.values[n]
    tensor_probs = sort_probs_iter.values[n].item()
    index = sort_probs_iter.indices[n]
    sort_prob_index_int = index.item()
    subreddit_string = id2label[sort_prob_index_int]
    print(f'{n+1}. {subreddit_string} ({tensor_probs * 100}%)')

In [ ]:
#print_out_top_x_probs(sort_prob, 10)

In [ ]:
phrase = input("Enter the phrase to predict the subreddit")
iterations = 5
print_out_top_x_probs(iterations, phrase)

Enter the phrase to predict the subredditgreen paint
Here are the top 10 most subreddits with similar


TypeError: 'builtin_function_or_method' object is not subscriptable

In [ ]:
# it's time to graph!

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy
    }


In [ ]:
import matplotlib.pyplot as plt

# this is fake data, replace with real data
epochs = 1
loss = [0.30, 0.24, 0.20]
f1_scores = [0.82, 0.85, 0.87]

plt.figure(figsize=(12, 5))

# Plotting loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Training Loss', marker='o')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Plotting F1 Score
plt.subplot(1, 2, 2)
plt.plot(epochs, f1_scores, label='F1 Score', marker='o', color='r')
plt.title('F1 Score per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()
